In [14]:
from z3 import *

def compute_tax_fixed(big_category, sub_category, quantity, free_vars=None):
    """
    計算固定稅額類別的應納稅額（針對水泥及油氣類），並支援指定參數為自由變數以求最小稅額。
    free_vars: list of strings, 指定參數名稱 (e.g. ['quantity']) 作為自由變數。
    回傳: best_tax, 原始固定稅額, final_params, differences
    """
    # Z3 優化器
    opt = Optimize()
    # Z3 變數
    quantity_z = Real('quantity')
    # 決定單位稅額
    if big_category == "cement":
        mapping = {"white":600, "bute1":320, "bute2":196, "other_cement":440}
    elif big_category == "oilgas":
        mapping = {"gasoline":6830, "diesel":3990, "kerosene":4250,
                   "aviation":610, "fuel_oil":110, "solvent_oil":720, "lpg":690}
    else:
        mapping = {}
    unit_tax = mapping.get(sub_category, 0)

    # 組建參數字典: param_name -> (z3_var, orig_value, [constraints])
    params = {
        'quantity': (quantity_z, quantity, [lambda v: v >= 0])
    }
    final_tax_z = Real('tax_amount')

    # 套用參數約束
    free_vars = free_vars or []
    for name, (var, val, constraints) in params.items():
        if name in free_vars:
            for c in constraints:
                opt.add(c(var))
        else:
            opt.add(var == val)
            for c in constraints:
                opt.add(c(var))

    # 稅額約束
    opt.add(final_tax_z == unit_tax * quantity_z)

    # 初始固定稅額
    if free_vars:
        opt.push()
        for name in free_vars:
            var, val, _ = params[name]
            opt.add(var == val)
        if opt.check() == sat:
            fixed_model = opt.model()
            fixed_tax = fixed_model[final_tax_z].as_long()
        else:
            fixed_tax = None
        opt.pop()
    else:
        if opt.check() == sat:
            fixed_tax = opt.model()[final_tax_z].as_long()
        else:
            fixed_tax = None

    print(f"初始固定稅額 (未放行自由變數) = {fixed_tax}")
    print(f"可調整變數: {free_vars}")

    # 開始優化
    best_tax = None
    iteration = 0
    while True:
        if opt.check() != sat:
            print(f"第 {iteration+1} 次最佳化: 無解，停止")
            break
        m = opt.model()
        current = m[final_tax_z].as_long()
        if iteration == 0 or current < best_tax:
            best_tax = current
            print(f"--- 第 {iteration+1} 次最佳化 ---")
            print(f"當前稅額 = {current}")
            diffs = {}
            for name, (var, orig, _) in params.items():
                try:
                    v_opt = m.eval(var).as_long()
                except:
                    v_opt = None
                if v_opt is not None and v_opt != orig:
                    diffs[name] = {'original': orig, 'optimized': v_opt, 'difference': v_opt - orig}
            if diffs:
                print("變更變數:")
                for k,v in diffs.items():
                    print(f"  {k}: 原={v['original']}, 優化後={v['optimized']}, 差異={v['difference']}")
            print("---------------------------")
        else:
            print(f"第 {iteration+1} 次最佳化: 未找到更優解，停止")
            break
        opt.add(final_tax_z < best_tax)
        iteration += 1

    # 整理結果
    final_params = {}
    differences = {}
    for name, (var, orig, _) in params.items():
        try:
            v_opt = m.eval(var).as_long()
        except:
            v_opt = None
        final_params[name] = {'value': v_opt, 'type': 'free' if name in free_vars else 'fixed'}
        if v_opt is not None and v_opt != orig:
            differences[name] = {'original': orig, 'optimized': v_opt, 'difference': v_opt - orig}

    return best_tax, fixed_tax, final_params, differences


def compute_tax_others(big_category, sub_category, quantity, assessed_price, is_electric=False, free_vars=None):
    """
    計算從價稅類別的應納稅額，並支援自由變數最佳化。
    free_vars: list of strings, e.g. ['quantity','assessed_price']
    回傳: best_tax, 固定稅額, final_params, differences
    """
    opt = Optimize()
    quantity_z = Real('quantity')
    assessed_price_z = Real('assessed_price')

    # 稅率決定
    if big_category == "rubber":
        rate = {"bus_truck":0.10, "other":0.15, "exempt":0.0}.get(sub_category,0)
    elif big_category == "beverage":
        rate = {"diluted_juice":0.08, "other":0.15, "exempt":0.0}.get(sub_category,0)
    elif big_category == "flat_glass":
        rate = {"regular":0.10, "exempt":0.0}.get(sub_category,0)
    elif big_category == "electrical":
        rate = {"refrigerator":0.13, "color_tv":0.13, "central_aircon":0.15,
                "non_central_aircon":0.20, "dehumidifier":0.15, "videorecorder":0.13,
                "non_portable_recordplayer":0.10, "portable_recordplayer":0.0,
                "tape_recorder":0.10, "stereo":0.10, "electric_oven":0.15}.get(sub_category,0)
    elif big_category == "vehicle":
        rate = {"compact_car":0.25, "large_car":0.30, "truck_bus_others":0.15,
                "motorcycle":0.17}.get(sub_category,0)
        if is_electric:
            rate /= 2.0
    else:
        rate = 0

    final_tax_z = Real('tax_amount')
    params = {
        'quantity': (quantity_z, quantity, [lambda v: v >= 0]),
        'assessed_price': (assessed_price_z, assessed_price, [lambda v: v >= 0])
    }
    free_vars = free_vars or []
    for name, (var, val, constraints) in params.items():
        if name in free_vars:
            for c in constraints:
                opt.add(c(var))
        else:
            opt.add(var == val)
            for c in constraints:
                opt.add(c(var))

    opt.add(final_tax_z == assessed_price_z * rate * quantity_z)

    if free_vars:
        opt.push()
        for name in free_vars:
            var, val, _ = params[name]
            opt.add(var == val)
        if opt.check() == sat:
            fixed_model = opt.model()
            fixed_tax = fixed_model[final_tax_z].as_long()
        else:
            fixed_tax = None
        opt.pop()
    else:
        if opt.check() == sat:
            fixed_tax = opt.model()[final_tax_z].as_long()
        else:
            fixed_tax = None

    print(f"初始固定稅額 (未放行自由變數) = {fixed_tax}")
    print(f"可調整變數: {free_vars}")

    best_tax = None
    iteration = 0
    while True:
        if opt.check() != sat:
            print(f"第 {iteration+1} 次最佳化: 無解，停止")
            break
        m = opt.model()
        current = m[final_tax_z].as_long()
        if iteration == 0 or current < best_tax:
            best_tax = current
            print(f"--- 第 {iteration+1} 次最佳化 ---")
            print(f"當前稅額 = {current}")
            diffs = {}
            for name, (var, orig, _) in params.items():
                try:
                    v_opt = m.eval(var).as_long()
                except:
                    v_opt = None
                if v_opt is not None and v_opt != orig:
                    diffs[name] = {'original': orig, 'optimized': v_opt, 'difference': v_opt - orig}
            if diffs:
                print("變更變數:")
                for k,v in diffs.items():
                    print(f"  {k}: 原={v['original']}, 優化後={v['optimized']}, 差異={v['difference']}")
            print("---------------------------")
        else:
            print(f"第 {iteration+1} 次最佳化: 未找到更優解，停止")
            break
        opt.add(final_tax_z < best_tax)
        iteration += 1

    final_params = {}
    differences = {}
    for name, (var, orig, _) in params.items():
        try:
            v_opt = m.eval(var).as_long()
        except:
            v_opt = None
        final_params[name] = {'value': v_opt, 'type': 'free' if name in free_vars else 'fixed'}
        if v_opt is not None and v_opt != orig:
            differences[name] = {'original': orig, 'optimized': v_opt, 'difference': v_opt - orig}

    return best_tax, fixed_tax, final_params, differences


# 範例執行
if __name__ == "__main__":
    # Example 1: 固定稅額 類別 cement, sub_category white, quantity=1000, 無自由變數
    print("\n=== Example 1: compute_tax_fixed ===")
    result1 = compute_tax_fixed("cement", "white", 1000, free_vars=['quantity'])
    print("Result:", result1)

    # Example 2: 從價稅 類別 vehicle, sub_category large_car, quantity=50, assessed_price=500000, 副電動
    print("\n=== Example 2: compute_tax_others ===")
    result2 = compute_tax_others("vehicle", "large_car", 50, 500000, is_electric=True, free_vars=['quantity'])
    print("Result:", result2)



=== Example 1: compute_tax_fixed ===
初始固定稅額 (未放行自由變數) = 600000
可調整變數: ['quantity']
--- 第 1 次最佳化 ---
當前稅額 = 0
變更變數:
  quantity: 原=1000, 優化後=0, 差異=-1000
---------------------------
第 2 次最佳化: 無解，停止
Result: (0, 600000, {'quantity': {'value': 0, 'type': 'free'}}, {'quantity': {'original': 1000, 'optimized': 0, 'difference': -1000}})

=== Example 2: compute_tax_others ===
初始固定稅額 (未放行自由變數) = 3750000
可調整變數: ['quantity']
--- 第 1 次最佳化 ---
當前稅額 = 0
變更變數:
  quantity: 原=50, 優化後=0, 差異=-50
---------------------------
第 2 次最佳化: 無解，停止
Result: (0, 3750000, {'quantity': {'value': 0, 'type': 'free'}, 'assessed_price': {'value': 500000, 'type': 'fixed'}}, {'quantity': {'original': 50, 'optimized': 0, 'difference': -50}})
